# 1. Instaling dependecies via pip comand

In [1]:
!pip install stable-baselines3[extra]

In [2]:
!pip install gym[all]

  Using cached mujoco-py-1.50.1.68.tar.gz (120 kB)
  Using cached box2d-py-2.3.8.tar.gz (374 kB)
  Running setup.py clean for box2d-py
  Running setup.py clean for mujoco-py
Failed to build box2d-py mujoco-py
    Running setup.py install for mujoco-py: started
    Running setup.py install for mujoco-py: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Gabriel Molina\Anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\GABRIE~1\\AppData\\Local\\Temp\\pip-install-rwvo8q7d\\box2d-py_b709089744574863aeefb89c5c5eff4e\\setup.py'"'"'; __file__='"'"'C:\\Users\\GABRIE~1\\AppData\\Local\\Temp\\pip-install-rwvo8q7d\\box2d-py_b709089744574863aeefb89c5c5eff4e\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\GABRIE~1\AppData\Local\Temp\pip-wheel-sycxftxi'
       cwd: C:\Users\GABRIE~1\AppData\Local\Temp\pip-install-rwvo8q7d\box2d-py_b709089744574863aeefb89c5c5eff4e\
  Complete output (16 lines):
  Using setuptools (version 58.0.4).
  running bdist_wheel
  running build
  running build

In [1]:
import gym
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Loading environment

In [2]:
env_name = "CartPole-v0"
env =  gym.make (env_name)

In [3]:
episodes = 5
for episode in range (1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print("Episode: {} , Score: {}".format(episode,score))
env.close()

Episode: 1 , Score: 14.0
Episode: 2 , Score: 26.0
Episode: 3 , Score: 14.0
Episode: 4 , Score: 9.0
Episode: 5 , Score: 22.0


# 2.1 Understanding the Env

https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [4]:
env.action_space
#this describes the space we have in the action within the env.
#Discrete(2) means we have two possibles discrete values: 0 and 1.

Discrete(2)

In [5]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

1

In [6]:
env.observation_space
#this describes the space we have with in the observation with in the env.
# Box means we have a list of lists, four values in each list, in a array of 4,0 and type flot32.

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [7]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([ 2.7027798e+00,  1.4961398e+38,  1.4780627e-03, -1.6779163e+38],
      dtype=float32)

# 3. Training RL Model

In [33]:
log_path = os.path.join('Training', 'Logs')

In [34]:
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [35]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_4
-----------------------------
| time/              |      |
|    fps             | 835  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 612        |
|    iterations           | 2          |
|    time_elapsed         | 6          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00912174 |
|    clip_fraction        | 0.108      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.686     |
|    explained_variance   | 0.000867   |
|    learning_rate        | 0.0003     |
|    loss                 | 5.64       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0157    |
|    value_loss           | 48.4       |
----------------------------------------
---------------------

# 4. Save and Reload Models

In [36]:
PPO_Path = os.path.join('Training', 'Saved Models', 'PPO_model_cartpole')
PPO_Path

'Training\\Saved Models\\PPO_model_cartpole'

In [37]:
model.save(PPO_Path)

In [38]:
del model

In [39]:
model = PPO.load(PPO_Path, env=env)

# 5. Evaluation

In [40]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [41]:
env.close()

# 6. Testing Model

In [42]:
episodes = 5
for episode in range (1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print("Episode: {} , Score: {}".format(episode,score))
env.close()

Episode: 1 , Score: [200.]
Episode: 2 , Score: [200.]
Episode: 3 , Score: [200.]
Episode: 4 , Score: [200.]
Episode: 5 , Score: [200.]


In [43]:
obs = env.reset() # reseting the initial conditions of environment
obs

array([[-0.04119328, -0.03246327,  0.00269216, -0.03548788]],
      dtype=float32)

In [44]:
action, _ = model.predict(obs) # predicting an action with the new set of conditions/observations

In [45]:
env.step(action) # step with the action taken and reword of 1, we succeed in mainting the pole in the upright position

(array([[-0.04184255, -0.22762372,  0.0019824 ,  0.25804323]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{}])

# 7. Logs in Tensorboard

In [46]:
training_log_path = os.path.join(log_path, 'PPO_4') #last succesfull model trained.
training_log_path

'Training\\Logs\\PPO_4'

In [47]:
!tensorboard --logdir = {training_log_path} # ! is a magic command. Issuing it here makes it execute in a command line.

TensorFlow installation not found - running with reduced feature set.
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--reuse_port BOOL] [--load_fast {false,auto,true}]
                   [--extra_data_server_flags EXTRA_DATA_SERVER_FLAGS]
                   [--grpc_creds_type {local,ssl,ssl_dev}]
                   [--grpc_data_provider PORT] [--purge_orphaned_data BOOL]
                   [--db URI] [--db_import] [--inspect] [--version_tb]
                   [--tag TAG] [--event_file PATH] [--path_prefix PATH]
                   [--window_title TEXT] [--max_reload_threads COUNT]
                   [--reload_interval SECONDS] [--reload_task TYPE]
                   [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data TYPE]
                   [--samples_per_plugin SAMPLES_PER_PLUGIN]
                   [--wh

# 8. Call Back to stop on desired reward (training stage)

In [51]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [52]:
save_path = os.path.join('Training', 'Saved Models')

In [54]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env,
                            callback_on_new_best=stop_callback,
                            eval_freq=10000,
                            best_model_save_path=save_path,
                            verbose=1
                            )

In [55]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [56]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_5
-----------------------------
| time/              |      |
|    fps             | 865  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 622          |
|    iterations           | 2            |
|    time_elapsed         | 6            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0093381405 |
|    clip_fraction        | 0.119        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.686       |
|    explained_variance   | -0.0023      |
|    learning_rate        | 0.0003       |
|    loss                 | 6.16         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0177      |
|    value_loss           | 46.5         |
----------------------------

C:\Users\Gabriel Molina\Anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


Eval num_timesteps=10000, episode_reward=191.40 +/- 17.20
Episode length: 191.40 +/- 17.20
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 191         |
|    mean_reward          | 191         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.009355215 |
|    clip_fraction        | 0.0698      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.614      |
|    explained_variance   | 0.383       |
|    learning_rate        | 0.0003      |
|    loss                 | 24          |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0147     |
|    value_loss           | 54.9        |
-----------------------------------------
New best mean reward!
------------------------------
| time/              |       |
|    fps             | 511   |
|    iterations      | 5     |
|    ti

   ## 8.1 Testing model with callback on rewardthreshold=200

In [57]:
episodes = 5
for episode in range (1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print("Episode: {} , Score: {}".format(episode,score))
env.close()

Episode: 1 , Score: [200.]
Episode: 2 , Score: [200.]
Episode: 3 , Score: [200.]
Episode: 4 , Score: [200.]
Episode: 5 , Score: [200.]


# 9 Changing Policies (network architecture of MLP)

In [67]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [68]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path, policy_kwargs={'net_arch': net_arch})

Using cpu device


In [69]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_6
-----------------------------
| time/              |      |
|    fps             | 579  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 396          |
|    iterations           | 2            |
|    time_elapsed         | 10           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0154761905 |
|    clip_fraction        | 0.215        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.682       |
|    explained_variance   | 0.00653      |
|    learning_rate        | 0.0003       |
|    loss                 | 2.86         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0226      |
|    value_loss           | 18.2         |
----------------------------

## 9.1 Testing with the new MLP architecture

In [70]:
episodes = 5
for episode in range (1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print("Episode: {} , Score: {}".format(episode,score))
env.close()

Episode: 1 , Score: [200.]
Episode: 2 , Score: [200.]
Episode: 3 , Score: [200.]
Episode: 4 , Score: [200.]
Episode: 5 , Score: [200.]


# 10. Alternate Algorithms

In [72]:
from stable_baselines3 import DQN

In [73]:
model_DQN = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [80]:
model_DQN.learn(total_timesteps=20000, log_interval=4)

Logging to Training\Logs\DQN_4
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.974    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 1969     |
|    time_elapsed     | 0        |
|    total_timesteps  | 55       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.936    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 2333     |
|    time_elapsed     | 0        |
|    total_timesteps  | 135      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.896    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 2319     |
|    time_elapsed     | 0        |
|    total_timesteps  | 218      |
----------------------------------
------------------------

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 3235     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2483     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 3238     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2540     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 3273     |
|    time_elapsed     | 0        |
|    total_timesteps  | 2685     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 3455     |
|    time_elapsed     | 1        |
|    total_timesteps  | 4750     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 3460     |
|    time_elapsed     | 1        |
|    total_timesteps  | 4837     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 3469     |
|    time_elapsed     | 1        |
|    total_timesteps  | 4925     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 3502     |
|    time_elapsed     | 2        |
|    total_timesteps  | 7134     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 3505     |
|    time_elapsed     | 2        |
|    total_timesteps  | 7213     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 3507     |
|    time_elapsed     | 2        |
|    total_timesteps  | 7290     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 3499     |
|    time_elapsed     | 2        |
|    total_timesteps  | 9491     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 3501     |
|    time_elapsed     | 2        |
|    total_timesteps  | 9553     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 440      |
|    fps              | 3507     |
|    time_elapsed     | 2        |
|    total_timesteps  | 9691     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 540      |
|    fps              | 3425     |
|    time_elapsed     | 3        |
|    total_timesteps  | 11859    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 544      |
|    fps              | 3421     |
|    time_elapsed     | 3        |
|    total_timesteps  | 11953    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 548      |
|    fps              | 3415     |
|    time_elapsed     | 3        |
|    total_timesteps  | 12025    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 3316     |
|    time_elapsed     | 4        |
|    total_timesteps  | 14247    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 652      |
|    fps              | 3317     |
|    time_elapsed     | 4        |
|    total_timesteps  | 14395    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 656      |
|    fps              | 3319     |
|    time_elapsed     | 4        |
|    total_timesteps  | 14529    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 3283     |
|    time_elapsed     | 5        |
|    total_timesteps  | 16897    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 760      |
|    fps              | 3275     |
|    time_elapsed     | 5        |
|    total_timesteps  | 16966    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 764      |
|    fps              | 3270     |
|    time_elapsed     | 5        |
|    total_timesteps  | 17018    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 3278     |
|    time_elapsed     | 5        |
|    total_timesteps  | 19471    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 868      |
|    fps              | 3278     |
|    time_elapsed     | 5        |
|    total_timesteps  | 19540    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 872      |
|    fps              | 3279     |
|    time_elapsed     | 5        |
|    total_timesteps  | 19611    |
----------------------------------
----------------------------------
| rollout/          

## 10.1 Testing DQN Algorithm trained

In [81]:
episodes = 5
for episode in range (1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action, _ = model_DQN.predict(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
        score += reward
    print("Episode: {} , Score: {}".format(episode,score))
env.close()

Episode: 1 , Score: [26.]
Episode: 2 , Score: [34.]
Episode: 3 , Score: [21.]
Episode: 4 , Score: [29.]
Episode: 5 , Score: [27.]


### as we can see this model do not performs very well with the above settings.

In [82]:
DQN_Path = os.path.join('Training', 'Saved Models', 'DQN_model_cartpole')

In [83]:
model_DQN.save(DQN_Path)